In [ ]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import numpy as np
import copy
import time
from tensorflow import keras
from tensorflow.keras import layers
import pyautogui
from PIL import Image
import time

font = cv2.FONT_HERSHEY_SIMPLEX
x1 = 50
y1 = 50
w = 200
p = 10
GuassianKernelSize = 7
ThreshValue = 5

fgbg = cv2.createBackgroundSubtractorMOG2()
takingData = False


def bgMask(img):
    fgmask = fgbg.apply(img)
    result = cv2.bitwise_and(img, img, mask=fgmask)
    return result

def guassianBlurFun(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (GuassianKernelSize,GuassianKernelSize), 100)
    return blur

def thresholdFun(img):
    ret, thresh = cv2.threshold(img, ThreshValue, 255, cv2.THRESH_BINARY)
    return thresh


def main():
    global takingData, colorFrame, font, x1, y1, w, p
    
    ## Load Your CNN model ##
    model = tf.keras.models.load_model('trained_model.h5')
    
    cap = cv2.VideoCapture(0) 
   
    while True:
        ret,frame = cap.read()
        frame = cv2.flip(frame, 1)
        cv2.rectangle(frame,(x1,y1),((250),(250)), (255,0,0), 2)
        
        roi = frame[y1:y1+w,x1:x1+w]
        
        res = bgMask(roi)

        blur = guassianBlurFun(res)
        
        thresh = thresholdFun(blur)
        cv2.imshow('thresh',thresh)

        
        #Contours
        contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        length = len(contours)
        maxArea = -1
        if length > 0:
            for i in range(length): 
                temp = contours[i]
                area = cv2.contourArea(temp)
                if area > maxArea:
                    maxArea = area
                    ci = i

            res = contours[ci]
            hull = cv2.convexHull(res)
            drawing = np.zeros(roi.shape, np.uint8)
            cv2.drawContours(drawing, [res], 0, (0, 255, 0), 2)
            cv2.drawContours(drawing, [hull], 0, (0, 0, 255), 3)
        cv2.imshow('main',drawing)
        

        img = Image.fromarray(drawing)
        img = img.resize((100,100))
        img_arr = np.array(img, np.float)
        img_arr = np.expand_dims(img_arr, axis =0)

        if takingData:
            colorFrame = (255, 0, 255)
  
            cv2.putText(frame, 'Data Taking: On', (5,20), font, 0.5, colorFrame, 2, 1)

            result = model.predict(img_arr)
            if result[0][0] == 1:
                predictions = 'Jump'
                pyautogui.press('space')
            else:
                predictions = 'Don"t Jump'
            cv2.putText(frame, 'Predection: {}'.format(predictions), (50,40), font, 0.5, colorFrame, 2, 1)
        else:
            colorFrame = (0,0,255)
            cv2.putText(frame, 'Data Taking: Off', (5,20), font, 0.5, colorFrame, 2, 1)

        cv2.imshow('Original',frame)

        key = cv2.waitKey(1) & 0xFF
        if key == 27 :
            break   
        elif key == ord('s'):
            takingData = not takingData   
            
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

<ipython-input-4-30e965f512c0>:85: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  img_arr = np.array(img, np.float)
